<a href="https://colab.research.google.com/github/ranieri-unimi/echo-malchiodi-2022/blob/main/echo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import os

os.environ["KAGGLE_USERNAME"] = "ranieriunimi"
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [164]:
ref = "bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows"

!kaggle datasets download $ref --unzip -p .

100% 12.6G/12.6G [01:20<00:00, 239MB/s]
100% 12.6G/12.6G [01:20<00:00, 169MB/s]


In [165]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [166]:
import pyspark
import findspark

In [167]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

# echo

set a number cap of tweets (0 to skip)

In [168]:
SAMPLE_SIZE = 0 * 1000

hadoop instance

In [169]:
from pyspark.sql import SparkSession

In [170]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

load the dataset

In [171]:
csv_list = [
    os.path.join(dirname, filename)
    for dirname, _, filenames in os.walk(".")
    for filename in filenames
    if filename.startswith("09")  # only September
]

In [172]:
df_list = [
    pd.read_csv(
        csv_file,
        engine='python',
        compression='gzip',
        encoding='utf-8',
        quoting=csv.QUOTE_ALL,
        usecols=lambda x: x in ["text","language","tweetid"]
        )
    for csv_file in csv_list
]

In [173]:
df = (pd
      .concat(df_list, axis=0)
      .reset_index(drop=True)
      .drop_duplicates(["tweetid"], keep='last')
      )

In [174]:
del df_list

In [175]:
df = df[df.language == "en"]

In [176]:
dl = df.text.tolist()
dl = random.sample(dl, SAMPLE_SIZE) if SAMPLE_SIZE else dl

In [177]:
rdd = sc.parallelize(enumerate(dl))

rdd.take(7)

[(0,
  'HIMARS work on 🇷🇺base in #Kupyansk/ east #Kharkiv https://t.co/6v9vEZU3MT'),
 (1,
  '@DarthPutinKGB Funny how that is! 🧐\n#UkraineWillWin\n#SlavaUkraïni\n#StandUpForUkraine\n#UkraineEU\n#ArmUkraineNow\n#RussiaIsLosing\n#russianinvasionofukraine\n#RussianWarCrimesInUkraine\n#RussianWarCriminals\n🛑#RussiaStateSponsorOfTerrorism\n#RussiaIsATerroristState\n🆘 #StopRussianAggression 🆘'),
 (2,
  'The #XiJinping Thought that has emerged as the guiding ideology to lead #China down the path of national rejuvenation exemplifies the ideology-authority relationship, states @am_i_t_kumar https://t.co/RqThnKSl6G'),
 (3,
  '@UNHumanRights says 38,000 🇺🇦children forcibly deported to 🇷🇺/ occupied territories…\n\nSTOP #Putin, NOW! ❌🚫 \n\n#RussiaUkraineWar #UkraineWillWin https://t.co/xyRtFn4WoP'),
 (4,
  '@EwanMacKenna @TheMiloExp @Goldfinger180 We are here to denazify this region and annex it. A referendum has already shown that 100% of the population agreed to join NAFO. \n#ArmUkraineNow #NAFOe

## data cleaning

In [178]:
def preprocess_tweet_text(tweet):

    # opt for a case insensite analysis
    tweet = tweet.lower()

    # remove URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet, flags=re.MULTILINE)

    # remove #hastags and @mentions
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)

    # translate puntctuation to spaces
    tweet = tweet.translate(
        str.maketrans(string.punctuation + "…’”“‘", " " * (len(string.punctuation) + 5))
    )

    # remove extra spaces
    tweet = " ".join(tweet.strip().split())

    return tweet

In [179]:
def shift_key_value(x):
    return (x[-1], x[0])

In [180]:
dataPipe = (
    rdd
    #    clean tweets
    .mapValues(preprocess_tweet_text)
    #    remove duplicates
    .map(shift_key_value)
    .reduceByKey(min)
    .map(shift_key_value)
    # nvm
    .filter(lambda x : 'ukrain' in x[-1])
)

dataPipe.take(7)

[(31, 'freeland has ukrainian roots her mother halyna khomiak is by origin'),
 (61,
  'with the international spotlight on the ukrainian crisis we are doing our part to highlight the arts produced by the former soviet country'),
 (125,
  'st petersburg local councilors call for putin to be tried for treason the new voice of ukraine'),
 (135,
  'biggest propaganda lie ever but these people want to join russia just watch a liberation video and see how tight the babushkas hug ukrainian soldiers oh wait i forgot they re propaganda videos'),
 (140,
  'in 2014 russians first attacked later did a voting amp 97 vote was given for staying with that time also gave nuclear threat or i will say they did the same what they doing now so voting will be done in eastern ukraine amp result will decide'),
 (165,
  'i really hope ukraine will get all its citizens back who were abducted by russia 😢🇺🇦🌻 maybe exchange them for pows the damage done to a beautiful country cannot be forgiven ïni 💖'),
 (168,
  '

## shingles

In [182]:
SH_LEN = 5

In [183]:
shigles = dataPipe.flatMap(
    # use a `SH_LEN` sliding window and list all shingles from each tweet `x`
    lambda x: [(x[0], x[-1][i : i + SH_LEN]) for i in range(len(x[-1]) - SH_LEN)]
).distinct()

shigles.take(7)

[(31, 'reela'),
 (31, 'd has'),
 (31, ' has '),
 (31, 'as uk'),
 (31, ' ukra'),
 (31, 'raini'),
 (31, 'an ro')]

In [184]:
import binascii

In [185]:
def hash_value(v):
    b = bytes(v, "utf-8") # from string to bytes
    h = binascii.crc32(b) # hashing to 4 bytes
    return h

In [186]:
charactMx = shigles.mapValues(hash_value).distinct()

charactMx.take(7)

[(31, 4096343853),
 (31, 2400758815),
 (31, 2879378705),
 (31, 1103104141),
 (31, 3531684517),
 (31, 3108453259),
 (31, 1553554569)]

## MinHashing

In [187]:
def RN(stop, start=1):
    """
        simple alias for random.randrange
        but `start` is set to 1 by default
    """
    return random.randrange(start, stop)


def RB(B=4):
    """
        Generate a odd number smaller than `B` bytes
    """
    return RN(2 ** (8 * B - 1), 0) * 2 + 1


def PF(x, scale, shift, base):
    """
        Permutation Function:
        Permute `x` by given parameters
    """
    return (scale * x + shift) % base


In [188]:
n_permfoo, b_bands = 120, 12

# approximation of similarity threshold
THRESHOLD = (1 / b_bands) ** (1 / (n_permfoo // b_bands)) 
THRESHOLD

0.7799771419043033

In [189]:
params = [ 
    {"shift": RB(), "scale": RB(), "base": 2 ** (8 * 4)}
    for _ in range(n_permfoo)
    # list of 120 linear congruential generators
]

params[:7]

[{'shift': 2109066455, 'scale': 3466147859, 'base': 4294967296},
 {'shift': 3254629851, 'scale': 1536077513, 'base': 4294967296},
 {'shift': 1309449615, 'scale': 1248170303, 'base': 4294967296},
 {'shift': 4063276759, 'scale': 3436356535, 'base': 4294967296},
 {'shift': 3084788331, 'scale': 2348595653, 'base': 4294967296},
 {'shift': 3757424555, 'scale': 2861559543, 'base': 4294967296},
 {'shift': 1341591263, 'scale': 100297629, 'base': 4294967296}]

In [190]:
def multi_enum(x):
    doc, shingle = x
    return [((doc, h), (shingle, h)) for h in range(n_permfoo)]
    # enumerate 120 times each key-value pair, for future computations

In [191]:
minHashSignMx = (
    charactMx.flatMap(multi_enum)
    .reduceByKey(
        lambda a, b: a if PF(a[0], **params[a[-1]]) < PF(b[0], **params[a[-1]]) else b
        # permute `PF` both `a` and `b`, and keep the one which return the smallest output
    )
    .mapValues(lambda v: v[0])
    # drop enumeration
)

minHashSignMx.take(7)
# (doc, h) , shingle

[((31, 1), 1472470462),
 ((31, 3), 1320991662),
 ((31, 5), 789266766),
 ((31, 7), 1553554569),
 ((31, 9), 3201754352),
 ((31, 11), 2233347650),
 ((31, 13), 80402308)]

## LSH

In [192]:
scale, shift = RN(b_bands), RN(b_bands)
bandmap = lambda x: PF(x, scale, shift, b_bands)

In [193]:
def band_expand(x):
  (doc, h), shingle = x
  # map each hash to band
  return ((doc, bandmap(h)), (h, shingle))

In [194]:
bandMx = (minHashSignMx #                              # (doc, h_hash) , shingle
          .map(band_expand) # map each hash to band    # (doc, band) , (h_hash , shingle) 
          .groupByKey().mapValues(list) # group        # (doc, band) , [(h_hash , shingle), (h_hash , shingle), ... ]
          .mapValues(lambda a : sorted(a, key=lambda x: x[0])) # as above, but sorted
          .mapValues(lambda a : [v for _, v in a] ) # drop `h_hash` # (doc, band) , [shingle, shingle, ...]
)

bandMx.take(7)

[((31, 9),
  [1472470462,
   80402308,
   942135126,
   403932167,
   3902633571,
   888318936,
   558010959,
   3646436640,
   4142514041,
   3886134158]),
 ((31, 7),
  [1320991662,
   2660742065,
   1168288677,
   888318936,
   2660742065,
   403932167,
   1674922228,
   4119603910,
   3093078239,
   66365117]),
 ((31, 5),
  [789266766,
   262548719,
   789266766,
   1168288677,
   1910779558,
   3646436640,
   366792667,
   4179268022,
   549095396,
   2233347650]),
 ((31, 3),
  [1553554569,
   4025010422,
   2305533283,
   1103104141,
   4096343853,
   2087081998,
   1910779558,
   1640797558,
   1413130087,
   3902633571]),
 ((31, 1),
  [3201754352,
   3265328535,
   2879378705,
   515114468,
   3531684517,
   2087081998,
   204025068,
   4164850696,
   4025010422,
   2433365378]),
 ((31, 11),
  [2233347650,
   2602829441,
   262548719,
   810810974,
   3265328535,
   3902633571,
   2305533283,
   1358253009,
   3093078239,
   4190168915]),
 ((61, 9),
  [1345723898,
   2855797699,

In [195]:
def band_reduce(x):
  (doc, band), a = x
  return ((hash(tuple(a)), band), doc) # hashing / bucketing shingle list

In [196]:
bandBuckets = (bandMx # (doc, band) , [shingle, shingle, ...]
    .map(band_reduce) # (sh_lst_hash, band), doc
    .groupByKey()
    .mapValues(list)  # (sh_lst_hash, band), [doc, doc, ...]
)

In [197]:
bucketList = bandBuckets.filter(lambda x: len(x[1]) > 1).map(lambda x : x[-1])

buckets = bucketList.collect()
buckets[:7]

[[339, 10136],
 [1352, 10164],
 [1583, 5294],
 [2593, 10577],
 [5536, 17124, 14215, 16969],
 [5536, 16298, 17124, 14215, 16969],
 [5536, 14215, 16969]]

## jaccard

collision may occurred, jaccard similarity has been computed to discard spourious couples

In [198]:
def add(a,b) : return a+b

In [199]:
adj_lst = {
    tuple(sorted([b[i], b[j]]))
    for b in buckets
    for i in range(len(b) - 1)
    for j in range(i + 1, len(b))
}

In [200]:
merry_keys = set(bucketList.reduce(add))

In [201]:
merry_shingles = shigles.filter(lambda x: x[0] in merry_keys).distinct()

In [202]:
cardinalities = merry_shingles.countByKey()
cardinalities

defaultdict(int,
            {339: 72,
             1352: 75,
             1583: 117,
             2593: 41,
             3342: 225,
             4408: 108,
             5437: 57,
             5536: 100,
             5876: 196,
             6322: 103,
             7219: 59,
             7856: 72,
             8403: 38,
             8516: 233,
             8769: 73,
             9008: 54,
             9253: 90,
             10164: 73,
             12097: 15,
             13142: 224,
             14024: 52,
             16298: 105,
             16897: 72,
             16919: 35,
             16976: 13,
             17124: 102,
             17378: 77,
             18451: 210,
             18687: 202,
             18892: 72,
             18925: 57,
             19542: 172,
             826: 78,
             1774: 13,
             2191: 212,
             2213: 56,
             2526: 77,
             2817: 112,
             3437: 41,
             3501: 112,
             3580: 61,
           

In [203]:
cardinality_of_intersections = (merry_shingles
    .cartesian(merry_shingles) # all combos
    .map(lambda x: ((x[0][0],x[1][0]) , (x[0][1], x[1][1])) ) # swap (k1,v1):(k2:v2) -> (k1,k2):(v1:v2)
    .filter(lambda x : x[0] in adj_lst) # not all combos
    .filter(lambda x : x[1][0] == x[1][1]) # intersected shingles
    .countByKey()
)
cardinality_of_intersections

defaultdict(int,
            {(1352, 10164): 64,
             (1583, 3501): 104,
             (3342, 13142): 179,
             (2593, 10577): 38,
             (4408, 6322): 100,
             (5536, 16298): 96,
             (5536, 17124): 98,
             (5437, 17665): 47,
             (5876, 19240): 193,
             (8769, 16897): 66,
             (8769, 9213): 65,
             (16298, 17124): 98,
             (826, 3706): 78,
             (2191, 18687): 200,
             (2526, 19802): 70,
             (2817, 16341): 108,
             (3437, 9591): 40,
             (3580, 8434): 55,
             (3991, 5327): 60,
             (5267, 9253): 90,
             (5179, 18451): 193,
             (5861, 16919): 32,
             (9213, 16897): 67,
             (16660, 17378): 72,
             (17844, 19542): 161,
             (4272, 6298): 106,
             (5460, 12836): 46,
             (6914, 14792): 13,
             (12049, 18541): 90,
             (14215, 16969): 96,
             (339, 

In [204]:
def jaccard(a, b):
    intersection = cardinality_of_intersections[(a,b)]
    union = cardinalities[a] + cardinalities[b] - intersection
    return (intersection / union)

In [205]:
jaccard_dict = { (a,b):jaccard(a,b) for a, b in adj_lst}

## results

In [206]:
import networkx as nx

In [207]:
G = nx.Graph()
G.add_edges_from([ k for k, v in jaccard_dict.items() if v > THRESHOLD]) # filter out
canditates = list(nx.connected_components(G)) # 

In [218]:
for v in random.sample(canditates, 3):
  for i in v:
    print(dl[i])
    print('---------------')
  print('___________________________________________________________________________________________')

Glory to Ukraine and it's Soldiers 🇺🇦
#UkraineRussiaWar https://t.co/IhBKMaEQrc
---------------
🇺🇦 Glory to Ukraine and it's Soldiers 🇺🇦
#UkraineRussiaWar https://t.co/IO2y8SnX5F
---------------
___________________________________________________________________________________________
@OzKaterji The world has to provide Ukraine with ALL necessary weapons to WIN the war #StandWithUkraine #russiaisanazistate #ArmUkraineNow #ATACMSForUkraine #F16ForUkraine #A10ForUkraine #UkrainianHeroes #SlavaUkraini 🇺🇦
---------------
@WarMonitor3 Great!! The world has to provide Ukraine with ALL necessary weapons to WIN the war #StandWithUkraine #AbramsForUkraine #ArmUkraineNow #ATACMSForUkraine #F16ForUkraine #A10ForUkraine #UkrainianHeroes #SlavaUkraini 🇺🇦
---------------
@ukraine_map @KyivIndependent Exactly! The world has to provide Ukraine with ALL necessary weapons to WIN the war #StandWithUkraine #AbramsForUkraine #ArmUkraineNow #ATACMSForUkraine #F16ForUkraine #A10ForUkraine #UkrainianHeroes #